In [7]:
import numpy as np

In [8]:
def rotate_coordinates(file_path, output_path, theta_x, theta_y, theta_z):
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Convert degrees to radians for rotations
    theta_x_rad = np.radians(theta_x)
    theta_y_rad = np.radians(theta_y)
    theta_z_rad = np.radians(theta_z)

    # Rotation matrix for counterclockwise rotation about the x-axis
    rotation_matrix_x = np.array([
        [1, 0, 0],
        [0, np.cos(theta_x_rad), -np.sin(theta_x_rad)],
        [0, np.sin(theta_x_rad), np.cos(theta_x_rad)]
    ])
    
    # Rotation matrix for counterclockwise rotation about the y-axis
    rotation_matrix_y = np.array([
        [np.cos(theta_y_rad), 0, np.sin(theta_y_rad)],
        [0, 1, 0],
        [-np.sin(theta_y_rad), 0, np.cos(theta_y_rad)]
    ])

    # Rotation matrix for counterclockwise rotation about the z-axis
    rotation_matrix_z = np.array([
        [np.cos(theta_z_rad), -np.sin(theta_z_rad), 0],
        [np.sin(theta_z_rad), np.cos(theta_z_rad), 0],
        [0, 0, 1]
    ])

    # Process each line containing atomic coordinates
    rotated_lines = []
    for line in lines[2:]:  # Skip first two lines (atom count and comment)
        parts = line.split()
        if len(parts) == 4:
            atom, x, y, z = parts
            coords = np.array([float(x), float(y), float(z)])

            # Apply rotations
            if theta_x != 0:
                coords = np.matmul(rotation_matrix_x, coords)
            if theta_y != 0:
                coords = np.matmul(rotation_matrix_y, coords)
            if theta_z != 0:
                coords = np.matmul(rotation_matrix_z, coords)

            # Create new line with rotated coordinates
            new_line = f"{atom}    {coords[0]:.5f}    {coords[1]:.5f}    {coords[2]:.5f}\n"
            rotated_lines.append(new_line)
            
    new_xyz_content = lines[:2] + rotated_lines
    
    with open(output_path, 'w') as file:
        file.writelines(new_xyz_content)

def translate_coordinates(file_path, output_path, translation_vector):
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    translation_vector = np.asarray(translation_vector)
    translated_lines = []
    for line in lines[2:]:  # Skip first two lines (atom count and comment)
        parts = line.split()
        if len(parts) == 4:
            atom, x, y, z = parts
            coords = np.array([float(x), float(y), float(z)])

            # Apply translation
            translated_coords = coords + translation_vector

            # Create new line with translated coordinates
            new_line = f"{atom}    {translated_coords[0]:.5f}    {translated_coords[1]:.5f}    {translated_coords[2]:.5f}\n"
            translated_lines.append(new_line)
            
    new_xyz_content = lines[:2] + translated_lines
    
    with open(output_path, 'w') as file:
        file.writelines(new_xyz_content)

def xyz_to_poscar(xyz_path, output_path, unit_cell_a, unit_cell_b, unit_cell_c):
    # Extracting the atomic symbols and positions from the xyz file
    with open(xyz_path, 'r') as file:
        lines = file.readlines()
        
    atom_data = []
    
    for line in lines[2:]:  # Skip the first two lines
        parts = line.split()
        if len(parts) == 4:
            symbol, x, y, z = parts
            atom_data.append((symbol, np.array([float(x), float(y), float(z)])))

    # Sort the atom data by atomic symbol
    atom_data.sort(key=lambda x: x[0])

    # Separate the symbols and positions
    sorted_symbols, sorted_positions = zip(*atom_data)

    # Convert the absolute positions to fractional coordinates relative to the unit cell
    # fractional_positions = [pos / unit_cell_dimensions for pos in sorted_positions]

    # Prepare the POSCAR header
    poscar_header = [
        "Generated POSCAR file\n",
        "1.0\n",  # Universal scaling factor
        f"{unit_cell_a[0]} {unit_cell_a[1]} {unit_cell_a[2]}\n",  # Unit cell vector a
        f"{unit_cell_b[0]} {unit_cell_b[1]} {unit_cell_b[2]}\n",  # Unit cell vector b
        f"{unit_cell_c[0]} {unit_cell_c[1]} {unit_cell_c[2]}\n"   # Unit cell vector c
    ]

    # Counting the number of atoms for each species
    unique_symbols = sorted(set(sorted_symbols))  # Sort to ensure consistent order
    atom_counts = [sorted_symbols.count(symbol) for symbol in unique_symbols]

    # Adding atomic species and count lines
    poscar_header.append(' '.join(unique_symbols) + '\n')
    poscar_header.append(' '.join(map(str, atom_counts)) + '\n')

    # Coordinate type (Direct for fractional coordinates)
    poscar_header.append("Cartesian\n")

    # Prepare POSCAR body with sorted fractional coordinates
    poscar_body = [f"{pos[0]:.16f} {pos[1]:.16f} {pos[2]:.16f}\n" for pos in sorted_positions]

    sorted_poscar_content = poscar_header + poscar_body

    # Write the new sorted POSCAR file
    with open(output_path, 'w') as file:
        file.writelines(sorted_poscar_content)


In [21]:
#rotate to align backbone to x-axis
# rotate_coordinates('PM6_poscar/PM6_4_no_side_relax.xyz','PM6_poscar/PM6_4_rotated.xyz', 0, 17.277, 3.6222)

In [17]:
#rotate again to align backbone to x-axis
# rotate_coordinates('PM6_poscar/PM6_4_rotated.xyz','PM6_poscar/PM6_4_rotated2.xyz', 0, 0.5314, 0)

In [18]:
#rotate to align backbone to x-axis
# translate_coordinates('PM6_poscar/PM6_1_rotated2.xyz','PM6_poscar/PM6_1_translated.xyz', (1.96796,  -14.53940,   -2.19935))

In [11]:
#convert to poscar
xyz_to_poscar('PM6_poscar/pm6_1_faceon_sidechain2.xyz', 'POSCARS/pi150/POSCAR_faceon_April24.vasp', 
              (21, 0, 0), (0, 19.5, 0), (0, 8.125, 3.6))

In [12]:
#convert to poscar
xyz_to_poscar('PM6_poscar/pm6_1_edgeon_sidechain2.xyz', 'POSCARS/pi150/POSCAR_edgeon_April24.vasp', 
              (19.5, 0, 0), (8.125, 3.6, 0), (0, 0, 21))

In [13]:
#convert to poscar
xyz_to_poscar('PM6_poscar/PM6_1_flat_edgeon.xyz', 'POSCARS/pi180_lam90/POSCAR_edgeon.vasp', (4.875, 0, 21), (9.75, 3.6, 0), (19.5, 0, 0))

In [5]:
# rotate to align backbone to z-axis
rotate_coordinates('PM6_poscar/pm6_1_faceon_bad_sidechain2.xyz', 'PM6_poscar/pm6_1_edgeon_sidechain2.xyz', 90, 0, 0)

In [17]:
[22.4,  0.,   0.]
[0.,   3.85, 9.75]
[0.,   0.,  19.5]

[0.0, 0.0, 19.5]

In [ ]:
[22.5  0.   0. ]
[ 0.    3.85 10.25]
[ 0.   0.  20.5]